# Exploring dataset

## Import datatset 

Major csv files from this competiton are train.csv, test.csv, sample_submission.csv, structures.csv. Thus, we will explore these files first.
    - train.csv: the training set
    - test.csv: the testing set
    - sample_submission.csv: a sample submission file in the correct format
    - structures.csv: molecular structure file, X Y Z
    

In [50]:
# import basic packages for ML
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt


In [65]:
# read all the major csv files
folder_name = 'champs-scalar-coupling/'
train = pd.read_csv(folder_name + 'train.csv')
test = pd.read_csv(folder_name + 'test.csv')
sample_submission = pd.read_csv(folder_name + 'sample_submission.csv')
structures = pd.read_csv(folder_name + 'structures.csv')

Additional dataset contains the following csv files:
    - dipole_moments.csv: contains the molecular electric dipole moments
    - magnetic_shielding_tensors.csv: contains the magnetic shielding tensors for all atoms in the molecules
    - mulliken_charges.csv: contains the mulliken charges for all atoms in the molecules.
    - potential_energy.csv: contains the potential energy of the molecules
    - scalar_coupling_contributions.csv: contains four types of contributions

In [66]:
# read additional files
dipole_moments = pd.read_csv(folder_name + 'dipole_moments.csv')
mag_shield_tensors = pd.read_csv(folder_name +
                                 'magnetic_shielding_tensors.csv')
mull_charges = pd.read_csv(folder_name + 'mulliken_charges.csv')
pot_energy = pd.read_csv(folder_name + 'potential_energy.csv')
sca_cou_contri = pd.read_csv(folder_name + 'scalar_coupling_contributions.csv')

## Merge datasets

In [67]:
# drop id column in train df
train.drop('id', axis=1, inplace=True)

In [68]:
# create distance features in dipole_moments
dipole_moments['distance'] = dipole_moments['X']+dipole_moments['Y']+dipole_moments['Z']
dipole_moments['distance_2'] = dipole_moments['distance'].apply(lambda x: x**2)

In [69]:
# merge train/test and structures with atom index
def map_atom_info(df, other_df, other_df_name, atom_idx):
    # merge train and structures
    if other_df_name == 'structures':
        df = pd.merge(df,
                      other_df,
                      how='left',
                      left_on=['molecule_name', f'atom_index_{atom_idx}'],
                      right_on=['molecule_name', 'atom_index'])
        
        df = df.drop('atom_index', axis=1)
        
        df = df.rename(
            columns={
                'atom': f'atom_{atom_idx}',
                'x': f'x_{atom_idx}',
                'y': f'y_{atom_idx}',
                'z': f'z_{atom_idx}'
            })
    # merge train and mag_shield_tensors
    elif other_df_name == 'mag_shield_tensors':
        other_df = other_df.drop(['YX', 'ZX', 'ZY'], axis=1)
        df = pd.merge(df,
                      other_df,
                      how='left',
                      left_on=['molecule_name', f'atom_index_{atom_idx}'],
                      right_on=['molecule_name', 'atom_index'])
        df.drop('atom_index', axis=1, inplace=True)
        df = df.rename(
            columns={
                'atom': f'atom_{atom_idx}',
                'XX': f'XX_{atom_idx}',
                'YY': f'YY_{atom_idx}',
                'ZZ': f'ZZ_{atom_idx}',
                'XY': f'XY_{atom_idx}',
                'YZ': f'YZ_{atom_idx}',
            })
    # merge train and mulliken_charges
    elif other_df_name == 'mull_charges':
        df = pd.merge(df,
                     other_df,
                     how='left',
                     left_on=['molecule_name', f'atom_index_{atom_idx}'],
                     right_on=['molecule_name', 'atom_index'])
        df.drop('atom_index', axis=1, inplace=True)
        df = df.rename(
            columns = {'mulliken_charge': f'mulliken_charge_{atom_idx}'}
        )
    
    else:
        print(f'{other_df} dataframe is not found')
        
    return df


In [70]:
# start merging
train = map_atom_info(train, structures, 'structures', 0)
train = map_atom_info(train, structures, 'structures', 1)
train = map_atom_info(train, mag_shield_tensors, 'mag_shield_tensors', 0)
train = map_atom_info(train, mag_shield_tensors, 'mag_shield_tensors', 1)
train = map_atom_info(train, mull_charges, 'mull_charges', 0)
train = map_atom_info(train, mull_charges, 'mull_charges', 0)

test = map_atom_info(test, structures, 'structures', 0)
test = map_atom_info(test, structures, 'structures', 1)
test = map_atom_info(test, mag_shield_tensors, 'mag_shield_tensors', 0)
test = map_atom_info(test, mag_shield_tensors, 'mag_shield_tensors', 1)
test = map_atom_info(test, mull_charges, 'mull_charges', 0)
test = map_atom_info(test, mull_charges, 'mull_charges', 1)

In [71]:
# merge train/test and structures without atom index
def merge_other_df(df, other_df, other_df_name):
    # merge train and pot_energy
    if other_df_name == 'pot_energy':
        df = pd.merge(df,
                     other_df,
                     how='left',
                     on='molecule_name')
    # merge train and dipole_moments
    elif other_df_name == 'dipole_moments':
        df = pd.merge(df,
                     other_df,
                     how='left',
                     on='molecule_name')
    else:
        print(f'{other_df} dataframe is not found')
        
    return df

In [72]:
# continue merging
train = merge_other_df(train, pot_energy, 'pot_energy')
train = merge_other_df(train, dipole_moments, 'dipole_moments')

test = merge_other_df(test, pot_energy, 'pot_energy')
test = merge_other_df(test, dipole_moments, 'dipole_moments')

# Data preprocessing

## exploring data insight

In [79]:
train.corr()['scalar_coupling_constant']

atom_index_0                0.018839
atom_index_1               -0.218989
scalar_coupling_constant    1.000000
x_0                        -0.006015
y_0                        -0.013119
z_0                         0.004807
x_1                         0.005132
y_1                         0.021189
z_1                        -0.007792
XX_0                       -0.024919
XY_0                       -0.000947
YY_0                       -0.042562
XZ_x                       -0.002161
YZ_0                        0.000404
ZZ_0                       -0.054809
XX_1                        0.223554
XY_1                        0.000740
YY_1                        0.221592
XZ_y                        0.000188
YZ_1                       -0.001670
ZZ_1                        0.237707
mulliken_charge_0          -0.025444
mulliken_charge_0          -0.025444
potential_energy           -0.017187
X                           0.000101
Y                           0.000315
Z                           0.002648
d

In [80]:
train.describe()

,atom_index_0,atom_index_1,scalar_coupling_constant,x_0,y_0,z_0,x_1,y_1,z_1,XX_0,...,YZ_1,ZZ_1,mulliken_charge_0,mulliken_charge_0,potential_energy,X,Y,Z,distance,distance_2
count,4.658147e+06,4.658147e+06,4.658147e+06,4.658147e+06,4.658147e+06,4.658147e+06,4.658147e+06,4.658147e+06,4.658147e+06,4.658147e+06,...,4.658147e+06,4.658147e+06,4.658147e+06,4.658147e+06,4.658147e+06,4.658147e+06,4.658147e+06,4.658147e+06,4.658147e+06,4.658147e+06
mean,1.335689e+01,5.883966e+00,1.592165e+01,1.058781e-01,-2.203691e-01,4.712888e-02,9.757184e-02,-3.956294e-01,7.768257e-02,2.962281e+01,...,2.856011e-01,1.110627e+02,1.221098e-01,1.221098e-01,-4.042052e+02,-1.881619e-02,8.021838e-02,2.280645e-01,2.894667e-01,8.073935e+00
std,3.267712e+00,4.993943e+00,3.494198e+01,1.759873e+00,2.121789e+00,1.564513e+00,1.479416e+00,1.767196e+00,1.255910e+00,3.033050e+00,...,1.766086e+01,5.816133e+01,4.474921e-02,4.474921e-02,3.715726e+01,2.123380e+00,1.613989e+00,1.015052e+00,2.826685e+00,1.538668e+01
min,0.000000e+00,0.000000e+00,-3.621860e+01,-9.234889e+00,-9.494160e+00,-9.134765e+00,-9.234889e+00,-9.254405e+00,-8.789131e+00,9.777800e+00,...,-2.660990e+02,-5.480430e+02,-3.086700e-02,-3.086700e-02,-7.146262e+02,-2.295800e+01,-9.252300e+00,-6.028500e+00,-2.337830e+01,0.000000e+00
25%,1.100000e+01,2.000000e+00,-2.549780e-01,-9.530924e-01,-1.836966e+00,-9.642128e-01,-7.927602e-01,-1.607567e+00,-6.696343e-01,2.761585e+01,...,-4.454300e+00,5.728035e+01,9.839700e-02,9.839700e-02,-4.244322e+02,-1.182200e+00,-8.477000e-01,-3.028000e-01,-1.447100e+00,5.470082e-01
50%,1.300000e+01,5.000000e+00,2.281130e+00,1.341212e-01,-2.979936e-01,4.972893e-03,4.660419e-02,-5.081915e-01,1.549763e-02,2.925070e+01,...,3.000000e-04,1.272380e+02,1.135030e-01,1.135030e-01,-4.032927e+02,-1.000000e-04,6.320000e-02,1.219000e-01,1.426000e-01,2.947746e+00
75%,1.600000e+01,8.000000e+00,7.390655e+00,1.227746e+00,1.753418e+00,1.035102e+00,9.682240e-01,2.882334e-01,8.926133e-01,3.121620e+01,...,4.814400e+00,1.534400e+02,1.287900e-01,1.287900e-01,-3.859742e+02,1.136900e+00,1.067400e+00,8.839000e-01,1.974900e+00,9.588932e+00
max,2.800000e+01,2.800000e+01,2.048800e+02,9.382240e+00,9.714469e+00,7.637578e+00,8.420893e+00,8.653665e+00,7.460225e+00,4.794440e+01,...,2.981170e+02,3.396940e+02,4.030940e-01,4.030940e-01,-4.052368e+01,2.187380e+01,1.300560e+01,6.880100e+00,2.153780e+01,5.465449e+02


In [81]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4658147 entries, 0 to 4658146
Data columns (total 33 columns):
molecule_name               object
atom_index_0                int64
atom_index_1                int64
type                        object
scalar_coupling_constant    float64
atom_0                      object
x_0                         float64
y_0                         float64
z_0                         float64
atom_1                      object
x_1                         float64
y_1                         float64
z_1                         float64
XX_0                        float64
XY_0                        float64
YY_0                        float64
XZ_x                        float64
YZ_0                        float64
ZZ_0                        float64
XX_1                        float64
XY_1                        float64
YY_1                        float64
XZ_y                        float64
YZ_1                        float64
ZZ_1                        flo

In [83]:
train.type.value_counts()

3JHC    1510379
2JHC    1140674
1JHC     709416
3JHH     590611
2JHH     378036
3JHN     166415
2JHN     119253
1JHN      43363
Name: type, dtype: int64

In [86]:
train.atom_0.value_counts()

H    4658147
Name: atom_0, dtype: int64

In [87]:
train.atom_1.value_counts()

C    3360469
H     968647
N     329031
Name: atom_1, dtype: int64